In [303]:
import numpy as np
import pandas as pd
import nltk
import random
from preprocess_tweet import PreprocessTweet

In [304]:
# read the text data
with open('twitter.txt', 'r') as f:
    data = f.read()

In [305]:
len(data)

3335477

In [306]:
print(data[:100])

How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way 


In [307]:
 def Preprocessing(data):
    """
    Preprocess the text corpus
    split the data by line breaks '\n'

    Parameters
    ----------
    data : str
        corpus in raw format

    Returns
    -------
    list_sentences :  list
        preprocessed list of list of tokens for each sentence.

    """

    # split by linebreaks
    sentences = data.split("\n")

    # strip whitespaces
    sentences = [s.strip() for s in sentences]

    #remove empty sentence
    sentences = [s for s in sentences if len(s) > 0]

    #tokenize, # and punctuation
    list_sentences = [s.lower().split(" ") for s in sentences]
    
    
    return list_sentences

In [308]:
tokenized_sentences = Preprocessing(data)

In [309]:
# split the data into train and test

random.seed(42)
random.shuffle(tokenized_sentences)

#80/20 train and test
lt = int(len(tokenized_sentences)*0.8)
train_data = tokenized_sentences[:lt]
test_data = tokenized_sentences[lt:]

print(f'size of train data:{len(train_data)}\nsize of test data:{len(test_data)}')

size of train data:38368
size of test data:9593


In [310]:
from collections import defaultdict, Counter
# get the count of all the words in the corpus

flat_list = [el for sentence in tokenized_sentences for el in sentence]

In [311]:
word_count = Counter(flat_list)
word_count.most_common(10)

[('the', 18863),
 ('to', 15647),
 ('i', 14263),
 ('a', 12254),
 ('you', 9732),
 ('and', 8660),
 ('for', 7716),
 ('in', 7469),
 ('is', 7268),
 ('of', 7221)]

In [312]:
word_count.get('the')


18863

#### Handle OoV(Out of Vocabulary) word

1. create a closed vocabulary
2. words that does not appear more frequently, assign then an unknows token <oov>


In [313]:
def create_closed_vocab(tokens, freq_threshold):
    """
    create a closed vocabulary for a given text corpus
    
    Parameters
    ----------
    tokens: list
        list of sentence tokens
    freq_threshold: int
        word frequemcy threshold
        
    Returns
    -------
    closed_vocab : list
        list of tokens having frequency greater than the threshold
        
    """
    
    closed_vocab = []
    for sentence_t in tokens:
        for word in sentence_t:
            if word_count.get(word) > freq_threshold:
                closed_vocab.append(word)
                
    return set(closed_vocab)
    

**Method to create a closed form vocab:**\
    1. get the word count for each token for th given corpus.\
    2. create a closed form vocab using training set.\
    3. replace the unseen word in training set (from closed from vocab) with <oov> token.\
    4. relace the unseen word of test set with <oov> token as well.

In [314]:
#get closed vocab using train set
closed_vocab =  create_closed_vocab(train_data, freq_threshold=2)


In [315]:
len(closed_vocab)

14905

In [316]:
def PreprocessTrainTest(data, vocab):
    """
    Handles unknown/missing word in training corpus with <oov> token
    
    parameters
    ----------
    data: list 
        list of list of tokens
    vocab: set
        set containing unique words in traiing data
        
    Returns
    -------
    replaced_sentence: list
        preprocessed list of list of tokens
    

    """
    #replace the token not in vocab with <oov>
    replaced_sentence = []
    for sentence in data:
        temp_sentence = []
        for token in sentence:
            if token not in vocab:
                temp_sentence.append('<oov>')
            else:
                temp_sentence.append(token)
        replaced_sentence.append(temp_sentence)
    
    return replaced_sentence
        
    

In [317]:
%%time
# preprocess train data
processed_train_data = PreprocessTrainTest(train_data, closed_vocab)

CPU times: user 117 ms, sys: 1.94 ms, total: 119 ms
Wall time: 118 ms


In [318]:
# processed_train_data[100:110]

In [319]:
%%time
#preproces test data
processed_test_data = PreprocessTrainTest(test_data, closed_vocab)

CPU times: user 44.7 ms, sys: 1.98 ms, total: 46.7 ms
Wall time: 45.5 ms


In [321]:
processed_test_data[3]

['thanks', 'for', 'the', 'follow', 'and', 'mention', '!']

# N-gram based language model

In [322]:
def create_n_gram(data, n=2, start_token= '<s>', end_token='<e>'):
    
    n_gram_dict = {}
    for sentence in data:
        # append n start token in the begining of each tokens
        tokens = sentence.copy()
        if n==1:
            tokens.insert(0, start_token)

        for i in range(n-1):
            tokens.insert(i, start_token)
        tokens.append(end_token)
        
        #create n gram count dictionary
        for i in range(len(tokens)):
            key = tuple(tokens[i:i+n])
            if len(key) == n:

                if key in n_gram_dict:
                    n_gram_dict[key]+=1

                else:
                    n_gram_dict[key]= 1
        
    return n_gram_dict

In [323]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]


In [324]:
sentences

[['i', 'like', 'a', 'cat'], ['this', 'dog', 'is', 'like', 'a', 'cat']]

In [325]:
create_n_gram(sentences, n=2)

{('<s>', 'i'): 1,
 ('i', 'like'): 1,
 ('like', 'a'): 2,
 ('a', 'cat'): 2,
 ('cat', '<e>'): 2,
 ('<s>', 'this'): 1,
 ('this', 'dog'): 1,
 ('dog', 'is'): 1,
 ('is', 'like'): 1}

In [326]:
create_n_gram(sentences, 1)

{('<s>',): 2,
 ('i',): 1,
 ('like',): 2,
 ('a',): 2,
 ('cat',): 2,
 ('<e>',): 2,
 ('this',): 1,
 ('dog',): 1,
 ('is',): 1}

In [327]:
## Estimate the porobaility
# use k smoothing to handle the missing N gram

def calculate_probabilities(previous_n_gram, vocab, prev_n_gram_count, n_gram_count, end_token ='<e>', oov_token='<oov>', k=1 ):
    """
    Parameters
    ----------

    previous_n_gram: tuple 
        sequenc of previous words of length n
    vocab: set
        set of unique word in training corpus
    prev_n_gram_count : dict
        dictionary for prev n-gram count
    n_gram_count : dict
        dictionary for n+1 gram count
        
    Return
    ------
    dictionary of joint probability of each word and previous n-gram word
    
    """
    # since start token ca not be the end of a n-gram
    #we didn't include start token in vocab
    vocab_new = vocab + [oov_token, end_token]
    probabilities = {}
    for word in vocab_new:
        joint_words = previous_n_gram +(word,)

        count = n_gram_count[joint_words] if joint_words in n_gram_count else 0
        prev_count = prev_n_gram_count[previous_n_gram] if previous_n_gram in prev_n_gram_count else 0
      
        #apply k smoothing
        prob = (count + k)/(prev_count + len(vocab)*k)
        probabilities[word] = prob
    
    return probabilities
    


In [328]:
prev_n_gram_count = create_n_gram(sentences, 1)
n_gram_count = create_n_gram(sentences, 2)
unique_words = list(set(sentences[0] + sentences[1]))
# unique_words
calculate_probabilities(('a',), unique_words.copy(),  prev_n_gram_count, n_gram_count)

{'dog': 0.1111111111111111,
 'is': 0.1111111111111111,
 'i': 0.1111111111111111,
 'a': 0.1111111111111111,
 'cat': 0.3333333333333333,
 'this': 0.1111111111111111,
 'like': 0.1111111111111111,
 '<oov>': 0.1111111111111111,
 '<e>': 0.1111111111111111}

In [329]:
sentences
unique_words

['dog', 'is', 'i', 'a', 'cat', 'this', 'like']

In [330]:
# lets plot them using matrix
def show_as_matrix(sentence, 
                   vocab, 
                   sentences,
                   start_token='<s>', 
                   end_token ='<e>', 
                   oov_token='<oov>',
                   n=2, 
                   k=1):
    """
    Prints the count matrix and probability matrix as dataframe for better visualization.
    
    """
  
    
    n_gram = []
    sentence = [start_token]*(n-1) + sentence

     
    vocab_new = list(set(vocab + [oov_token, end_token]))
    
    prev_n_gram_count = create_n_gram(sentences, n-1)
    n_gram = list(set(prev_n_gram_count.keys())) 
    n_gram_count = create_n_gram(sentences, n)
    
    count_matrix = np.zeros((len(n_gram), len(vocab_new)))
    prob_matrix = np.zeros((len(n_gram), len(vocab_new)))
    

    
    for i in range(len(n_gram)):
        for j in range(len(vocab_new)):
            n_token = n_gram[i] + (vocab_new[j],)
            count_n_token = n_gram_count[n_token] if n_token in n_gram_count else 0
            count_matrix[i,j] = count_n_token
            prob_matrix[i,j] = calculate_probabilities(n_gram[i], vocab_new,prev_n_gram_count, n_gram_count)[vocab_new[j]]
    
    count_matrix = pd.DataFrame(count_matrix, index=n_gram, columns=vocab_new)
    prob_matrix = pd.DataFrame(prob_matrix, index=n_gram, columns=vocab_new)
    
    return count_matrix, prob_matrix
    

In [400]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))


In [401]:
cm, pm = show_as_matrix(unique_words, unique_words,sentences, n=3 )

In [402]:
cm

,dog,is,<e>,i,a,cat,this,<oov>,like
"(<s>, i)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
"(<s>, this)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(dog, is)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
"(i, like)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(is, like)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(this, dog)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(cat, <e>)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [403]:
pm

,dog,is,<e>,i,a,cat,this,<oov>,like
"(<s>, i)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000
"(<s>, this)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(dog, is)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000
"(i, like)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(is, like)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(this, dog)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(cat, <e>)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(a, cat)",0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(like, a)",0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909


In [336]:
# Now we have the probability matrix and count matrix, Now we are ready to develop a autocomplete model.

In [337]:
import math

In [338]:
def perplexity_of_sentence(sentences, sentence, vocab, n = 2, start_token ='<s>', end_token ='<e>'):
    """
    parameters
    ----------
    sentences: list of list of sentence tokens
        list of words of sentence
    sentence :  list 
        list of toknes whose pp is to be calculated
    vocab :  list 
        list of unique words in the corpus
    n : int
        expected n gram
        
    Returns
    -------
    pp : float
        perplexity score for a given sentence
        
    """
#     sentence = sentences[1]
    prev_n_gram_count = create_n_gram(sentences, n-1)
    n_gram_count = create_n_gram(sentences, n)
    
    
    sentence = [start_token]*(n-1) + sentence +[end_token]
    sentence = tuple(sentence)
    
    N = len(sentence)
    
    pp_prob = 1
    for i in range(n-1, N):
        n_gram = sentence[i-n+1:i]
        
        word = sentence[i]
        
        #include start and end token in calculating prob
        # count the start and end token in sequence lenght also
        prob = calculate_probabilities(n_gram, vocab, prev_n_gram_count, n_gram_count)[word]
        pp_prob+=math.log((1/prob))
    
    
    pp = pp_prob**(float(1/N))
    return pp
    

In [339]:
test_sentence = ['i', 'like', 'a', 'dog']
perplexity_of_sentence(sentences, test_sentence, unique_words )

1.4492589235968028

In [340]:
def predict_nextword(sentences, sentence, vocab, n = 2, start_token ='<s>', end_token ='<e>', begins_with='d', suggestion_freq=2):
    """
    parameters
    ----------
    sentences: list of list of sentence tokens
        list of words of sentence
    sentence :  list 
        list of toknes whose pp is to be calculated
    vocab :  list 
        list of unique words in the corpus
    n : int
        expected n gram
        
    Returns
    -------
    a list of predicted k words where k is suggestion_freq
    
    """
    
    #get previous n-1 word
    sentence = tuple(sentence)
    prev_word = sentence[-(n-1):]
    
    new_vocab = list(set(vocab + [end_token]))
    prev_n_gram_count = create_n_gram(sentences, n-1)
    n_gram_count = create_n_gram(sentences, n)
    
    
    # get the probability for prev word and next candidate word
    prob = calculate_probabilities(prev_word, new_vocab, prev_n_gram_count, n_gram_count)
    prob =  {k: v for k, v in sorted(prob.items(), reverse=True,  key=lambda item: item[1])}
    
    
    
    res = []
    for k, v in prob.items():
        if len(begins_with) > 0:
            if k.startswith(begins_with):
                res.append({k:v})
        else:
            res.append({k:v})
                
                
    return res[:suggestion_freq]
    print(res[:suggestion_freq])

In [341]:
sentences[0][:-1]

['i', 'like', 'a']

In [342]:
predict_nextword(sentences, sentences[0][:-1],unique_words, begins_with='d' )

[{'dog': 0.1}]

In [382]:
def autocompleteSentence(sentences, sentence, vocab, n = 2, start_token ='<s>', end_token ='<e>'):
    """
    parameters
    ----------
    sentences: list of list of sentence tokens
        list of words of sentence
    sentence :  list 
        list of toknes whose pp is to be calculated
    vocab :  list 
        list of unique words in the corpus

        
    Returns
    -------
        complete sentence based on n-gram probability
    """
    sentence = sentence.copy()
    curr_word = start_token
    while curr_word != end_token:
        prev_word = list(sentence[-(n-1):])

        curr_word = list(predict_nextword(sentences, prev_word, vocab, n, begins_with='', suggestion_freq=1)[0])[0]      

        sentence.append(curr_word)
        
    return " ".join(sentence[:-1])
    

In [383]:
sentence = sentences[1][:-2]
sentence

['this', 'dog', 'is', 'like']

In [387]:

# for n in range(2, 4):
#     print(n)
pred = autocompleteSentence(sentences, sentence, unique_words, n=3 )
perplexity_of_sentence(sentences, pred.split(" "), unique_words, n=3)

1.2914802650430801

In [394]:
# check the perplexity score for different n gram predicted sentence
for n in range(2, 6):
    
    pred = autocompleteSentence(sentences, sentence, unique_words, n=n )
    score = perplexity_of_sentence(sentences, pred.split(" "), unique_words, n=n)
    
    print(f"for {n}-gram, PP: {score}")


for 2-gram, PP: 1.3328333831401098
for 3-gram, PP: 1.2914802650430801
for 4-gram, PP: 1.2624417182398258
for 5-gram, PP: 1.2390810140561817


## Now let's test on our test dataset

In [396]:
sentence = processed_test_data[7][:-3]
autocompleteSentence(processed_train_data, sentence, list(closed_vocab), n=3)

'<oov> my mom just made my day (: ! she said that shes getting shirts and braclets for me to <oov>'

In [399]:
processed_test_data[7]

['<oov>',
 'my',
 'mom',
 'just',
 'made',
 'my',
 'day',
 '(:',
 '!',
 'she',
 'said',
 'that',
 'shes',
 'getting',
 'shirts',
 'and',
 'braclets',
 'for',
 'me',
 'on',
 'christmas',
 '<oov>']